In [1]:
import sys
sys.path.append('../')
from rocky.core import rocky
from rocky.triangle import Triangle
from rocky.model_selection.TriangleTimeSeriesSplit import TriangleTimeSeriesSplit
import pandas as pd
import numpy as np

In [2]:
t = Triangle.from_taylor_ashe()
t.df

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Create a Rocky object
rky = rocky()

# add triangle
# rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
# rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.


In [22]:
cv = TriangleTimeSeriesSplit(triangle=rky.icrfs.tri, model_type='loglinear', loglinear_grid=None, X=rky.icrfs.GetX(), y=rky.icrfs.GetY(), n_splits=3)
cv.SetParameterGrid('loglinear', alpha=cv.loglinear_grid['alpha'], l1_ratio=[0.1])
cv.grid

{'alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
        1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
        2.6, 2.7, 2.8, 2.9, 3. ]),
 'l1ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
        0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
 'max_iter': [100000]}

{'alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
        1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
        2.6, 2.7, 2.8, 2.9, 3. ]),
 'l1ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
        0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]),
 'max_iter': [100000]}

In [15]:
gen = cv.GetFittedStatistics(model_params={'alpha':0.1, 'l1_ratio':0.01})
gen

,n_run,train_mse,test_mse,train_mae,test_mae,train_mape,test_mape,train_msle,test_msle
0,1,0.268450,0.523548,0.434819,0.574772,1.923866,1.727333,NaN,NaN
1,2,0.199043,0.615443,0.368528,0.585676,0.903767,2.080786,NaN,NaN
2,3,0.110907,1.168970,0.276054,0.806146,1.093840,1.322248,NaN,NaN


In [ ]:
# set hyperparameters
# rky.icrfs.SetHyperparameters(alpha=0.025, l1_ratio=0.1)
# rky.icrfs.TuneFitHyperparameters()

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/4.8495 2/2.4705 3/2.2253 4/1.8014 5/1.7065 6/1.4461 7/1.3691 8/1.2073 9/1.1492 10/1.0221 11/0.9694 12/0.8668 13/0.8187 14/0.7342 15/0.6902 16/0.6197 17/0.5794 18/0.5197 19/0.4829 20/0.4320 21/0.3984 22/0.3488 23/0.3139 24/0.2747 25/0.2455 26/0.2146 27/0.1904 28/0.1661 29/0.1463 30/0.1271 31/0.1110 32/0.0952 33/0.0816 34/0.0677 35/0.0556 36/0.0435 37/0.0327 38/0.0220 39/0.0122 40/0.0028 

In [ ]:
rky.icrfs.LassoFeatureImportance()

,0.0000,0.0025,0.0050,0.0075,0.0100,0.0125,0.0150,0.0175,0.0200,0.0225,...,0.4025,0.4050,0.4075,0.4100,0.4125,0.4150,0.4175,0.4200,0.4225,0.4250
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.555341,0.437116,0.381993,0.340483,0.304968,0.269831,0.233720,0.202262,0.175553,0.147187,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
2,0.401034,0.219079,0.155947,0.115349,0.081058,0.048525,0.016292,0.000000,0.000000,0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
3,0.183097,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
4,0.303462,0.054724,0.011176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.378221,0.071433,0.017413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,0.489392,0.146359,0.091176,0.039039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,0.632052,0.233083,0.175286,0.108900,0.047392,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,0.240184,-0.063399,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,0.000140,-0.246843,-0.055387,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [ ]:
rky.icrfs.tri.ult()

Accident Period
2001-01-01    3901463.0
2002-01-01    5433719.0
2003-01-01    5378826.0
2004-01-01    5297906.0
2005-01-01    4858200.0
2006-01-01    5111171.0
2007-01-01    5660771.0
2008-01-01    6784799.0
2009-01-01    5642266.0
2010-01-01    4969825.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [ ]:
rky.icrfs.Ultimate()

accident_period
2001-01-01    3232494.0
2002-01-01    4341155.0
2003-01-01    4162721.0
2004-01-01    4173190.0
2005-01-01    4282171.0
2006-01-01    4471540.0
2007-01-01    5070201.0
2008-01-01    5579252.0
2009-01-01    5230096.0
2010-01-01    5539242.0
Name: Log-linear Ultimate, dtype: float64

In [ ]:
rky.icrfs.TrendPlot()

In [ ]:
rky.icrfs.FitPlot('acc')

In [ ]:
rky.icrfs.ResidualPlot(color_col='acc')

$\text{AIC} = 2k - 2l$

In [ ]:
# k - number of parameters:
rky.icrfs.GetP()

37

In [ ]:
# l - loglikelihood:
# log(likelihood)


In general

Objective function of elastic net regularization:

$$
\min_{\beta} \frac{1}{2n} \sum_{i=1}^n \left( y_i - \beta_0 - \mathbf{x}_i^T\beta \right)^2 + \alpha \lambda \|{\beta}\|^1 + \frac{\alpha(1-\lambda)}{2} \|\beta\|_2^2
$$

Here $\alpha$ is the strength of the regularization, $\lambda$ is the portion of the regularization that is $\ell_1$-norm, and $k$ is the number of parameters in the model.

